<a href="https://colab.research.google.com/github/HerrSorokin/DSL/blob/main/Job_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To clean our grammar, we find all productive and reachable non-terminals and then take away all needless. 

First, we find productive non-terminals by looking through every non-terminal and its product rules, noting those NT whose right side of the rule contains only tokens or other productive non-terminals.

Second, we find reachable NT. The first reachable NT is main NT, then we add all NT contained in its rules to the set of reachable and repeat this operation with the set of reachable while it grows.

Third, we compare the original set of NT with the union of productive and reachable NT and include only productive NT with rules containing just reachable  NT.

In [1]:
def clean_external(gr):

  toks = gr['toks']
  vars = gr['vars']
  n_extr = set()
  
  #вспомогательная функция для определения продуктивности нетерминала
  def is_non_external(definition):
    return any(map(is_rule_right, definition))

  #вспомогательная функция для анализа правой части правила на непродуктивные нетерминалы
  def is_rule_right(rule):
    return all(map((lambda part: part in toks.union(n_extr)), rule))

  #вспомогательная функция для "очищения" правил от непродуктивныъ нетерминалов
  def clean_definition(definition):
    new_definition = []
    for rule in definition:
        if is_rule_right(rule):
          new_definition.append(rule)
    return new_definition


  #ищем продуктивные нетерминалы
  new_n_extr= True 
  while new_n_extr:
    new_n_extr = False
    for nterm, definition in vars.items():
      if nterm not in n_extr and is_non_external(definition):
        n_extr.add(nterm)
        new_n_extr = True


  #ищем достижимые нетерминалы
  finded = set(gr['hvar'])
  it_grows = True
  while it_grows:
    it_grows = False
    for nterm, definition in vars.items():
      if nterm in finded:
        for rule in definition:
          new_finded = finded.union(set(filter(lambda part: part in vars, rule)))
          it_grows = not (len(finded) == len(finded))
          finded = new_finded

  #получаем множество сторонних нетерминалов
  n_extr = n_extr.union(finded)
  
  #удаляем из грамматики все посторонних нетерминалы и правила в которых они содержатся)
  new_vars = dict()
  for nterm, definition in vars.items():
    if nterm in n_extr:
      new_definition = clean_definition(definition)
      if nterm not in new_vars.keys() and new_definition:
        new_vars[nterm] = []
        new_vars[nterm] = new_definition
      
  gr['vars'] = new_vars

  return gr



To find all vanishing NTs we look through every non-terminal and its product rules and if we find rule like α->ε or α->β, when β->>ε , we are marking this NT as vanishing and repeat while we find new vanishing NTs 

In [4]:
def vanishing(gr, empt):
  toks = gr['toks']
  vars = gr['vars']
  vanishing_vars = set() 
  
  #вспомогательная функция для определения исчезаемости нетерминала
  def is_vanishing(definition):
    for rule in definition: 
       if all(map((lambda way: way == empt or way in vanishing_vars), rule)): return True
    return False

  #ищем исчезающие нетерминалы
  new_vanishing_flag = True 
  while new_vanishing_flag:
    new_vanishing_flag = False
    for nterm, definition in vars.items():
      if nterm not in vanishing_vars and is_vanishing(definition) :
        vanishing_vars.add(nterm)
        new_vanishing_flag = True
          
  return vanishing_vars.difference(gr['hvar']) #исключаем главный нетерминал

  

In [5]:
gr = {
    'toks': set( [
        (0, 'e'), 
        (1, 'a'), 
        (2, 'b'), 
        (3, 'c'),
        (4, 'd'),
        (5, 'f'),
    ] ),

    'vars': {
        'A' : [['A', (1, 'a')], 
               ['B', 'F'],
               ['C', (3, 'c'), 'D']],
        'B' : [['D', (1, 'c')]],
        'C' : [['B'],
               [(0, 'e')]],
        'D' : [['B', (4, 'd')],
               ['C', (2, 'b')]],
        'F' : [[(0, 'e')],
               [(2, 'b'), 'B'],
               ['C']],
        'S' : [['A']]
    },
    'hvar': 'S'
}

print("Grammar without unusefull(external) non-terminals: \n", clean_external(gr))
print("Vanishing non-terminals: ", vanishing(gr, (0, 'e')))

Grammar without unusefull(external) non-terminals: 
 {'toks': {(4, 'd'), (5, 'f'), (0, 'e'), (3, 'c'), (2, 'b'), (1, 'a')}, 'vars': {'A': [['A', (1, 'a')], ['C', (3, 'c'), 'D']], 'C': [[(0, 'e')]], 'D': [['C', (2, 'b')]], 'F': [[(0, 'e')], ['C']], 'S': [['A']]}, 'hvar': 'S'}
Vanishing non-terminals:  {'F', 'C'}


Result:
Grammar without unusefull(external) non-terminals: {
  
  'toks': {(2, 'b'), (5, 'f'), (0, 'e'), (4, 'd'), (1, 'a'), (3, 'c')},
   
   'vars': {'A': [['A', (1, 'a')], ['C', (3, 'c'), 'D']], 'C': [[(0, 'e')]] 'D': [['C', (2, 'b')]], 'F': [[(0, 'e')], ['C']], 'S': [['A']]},
   
   'hvar': 'S'}
   
Vanishing non-terminals:  {'F', 'C'}